### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = '15min'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.5 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0    # percentage point specifying the validation set end point (1.0 means no test set)
# basically this is the data with the values above, which are like sliders determining the layout
# [|0.0| ................... train .............. |0.5| ....................... val ..................... |1.0| no test ]

In [ ]:
def backtest(clf):
    s = MLClassifierStrategy(clf, list(data.filter(like='X')))
    equity, pf, trades = backtest_ml_strategy(s, data)
    plt.plot(equity);
    plt.xlabel('Bar #')
    plt.ylabel('Profit')
    print(f'Profit factor: {pf}, trades: {len(trades)}')
    return equity, pf, trades

### ------------------------------------------------------------------------------------------------------------

### LogisticRegression

In [ ]:
# Train LogisticRegression classifier on train data
clf = train_classifier(LogisticRegression, data)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### XGBClassifier

In [ ]:
# Train XGBClassifier classifier on train data
clf = train_classifier(XGBClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### LGBMClassifier

In [ ]:
# Train LGBMClassifier classifier on train data
clf = train_classifier(LGBMClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### RandomForestClassifier

In [ ]:
# Train RandomForestClassifier classifier on train data
clf = train_classifier(RandomForestClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### ExtraTreesClassifier

In [ ]:
# Train ExtraTreesClassifier classifier on train data
clf = train_classifier(ExtraTreesClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### CatBoostClassifier

In [ ]:
# Train CatBoostClassifier classifier on train data
clf = train_classifier(CatBoostClassifier, data, verbose=False)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### LogisticRegression + HPO

In [ ]:
# Train LogisticRegression classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
best_hyperparams = optimize_model(LogisticRegression(), 'LogisticRegression', {"C": hp.loguniform("C", -5, 2)},
                                  X_train, y_train, max_evals=120)
clf = train_classifier(LogisticRegression, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### XGBClassifier + HPO

In [ ]:
# Train XGBClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
best_hyperparams = optimize_model(XGBClassifier(), 'XGBClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                   "learning_rate": hp.loguniform("learning_rate", -5, 0), "max_depth": hp.choice("max_depth", range(2, 11))},
                                  X_train, y_train, max_evals=10)
clf = train_classifier(XGBClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### LGBMClassifier + HPO

In [ ]:
# Train LGBMClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
best_hyperparams = optimize_model(LGBMClassifier(), 'LGBMClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                  "learning_rate": hp.loguniform("learning_rate", -5, 0),
                                  "max_depth": hp.choice("max_depth", range(2, 11))},
                                  X_train, y_train, max_evals=10)
clf = train_classifier(LGBMClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### RandomForestClassifier + HPO

In [ ]:
# Train RandomForestClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
best_hyperparams = optimize_model(RandomForestClassifier(), 'RandomForestClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                   "max_depth": hp.choice("max_depth", range(2, 21))},
                                  X_train, y_train, max_evals=10)
clf = train_classifier(RandomForestClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### ExtraTreesClassifier + HPO

In [ ]:
# Train ExtraTreesClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
best_hyperparams = optimize_model(ExtraTreesClassifier(), 'ExtraTreesClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                   "max_depth": hp.choice("max_depth", range(2, 21))},
                                  X_train, y_train, max_evals=10)
clf = train_classifier(ExtraTreesClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### CatBoostClassifier + HPO

In [ ]:
# Train CatBoostClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
best_hyperparams = optimize_model(CatBoostClassifier(verbose=False), 'CatBoostClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                   "learning_rate": hp.loguniform("learning_rate", -5, 0),
                                   "max_depth": hp.choice("max_depth", range(2, 11))},
                                  X_train, y_train, max_evals=10)
best_hyperparams['verbose'] = False
clf = train_classifier(CatBoostClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = backtest(clf)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------